In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('./data/dataset.csv')
df

,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),収率(%)
0,1,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,24.0,0.0,0.0,81.0
1,2,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,12.0,0.0,0.0,14.0
2,3,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,74.0
3,4,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,12.0,0.0,0.0,15.0
4,5,[*]C1=CC=CC=C1,H,OC1=CC(OC)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...
201,2l-c,[*]C1=CC=CC=C1,Ph,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,72.0,0.0,0.0,92.0
202,2m,[*]C1=CSC=C1,C,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,83.0
203,2n,NaN,NaN,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,53.0
204,2o,NaN,NaN,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,25.0


# 前処理
1. 欠損値削除(R1-,R2,organocatalyst,yield)
2. under = Ar削除(1個)
3. R2 = Ph削除(3個)
4. 触媒促進剤使ってるデータも削除
5. rtを23度に置換
6. カテゴリ値をダミー変数に変換

In [5]:
# 基質、触媒、収率で欠損がある場合は削除
df.dropna(subset=['R1-', 'R2', 'organocatalyst', '収率(%)'], inplace=True)

In [6]:
print('欠損値合計', df.isnull().sum())
na_row = df.isnull().any(axis=1)
df.loc[na_row, :]

欠損値合計 entry                   0
R1-                     0
R2                      0
organocatalyst          0
organocatalyst(mol%)    0
solvent                 0
under                   0
temp(℃)                 0
time(h)                 0
Cu(OAc)2(mol%)          0
AcOH(mol%)              0
収率(%)                   0
dtype: int64


,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),収率(%)


In [7]:
# Arを削除(1 data)
df = df[df['under'] != 'Ar']
df['under'].value_counts()

O2     110
air     44
Name: under, dtype: int64

In [8]:
# R2 = Ph削除(3 data)
df = df[df['R2'] != 'Ph']
df['R2'].value_counts()

H    99
C    52
Name: R2, dtype: int64

In [9]:
# 触媒促進剤データ削除
df = df[df['Cu(OAc)2(mol%)'] == 0]
df = df[df['AcOH(mol%)'] == 0]
print(df.columns)
print(df.shape)

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)',
       'solvent', 'under', 'temp(℃)', 'time(h)', 'Cu(OAc)2(mol%)',
       'AcOH(mol%)', '収率(%)'],
      dtype='object')
(123, 12)


In [10]:
# 触媒カラム削除
df.drop(columns=['Cu(OAc)2(mol%)','AcOH(mol%)'],inplace=True)

df.columns

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)',
       'solvent', 'under', 'temp(℃)', 'time(h)', '収率(%)'],
      dtype='object')

In [11]:
# rtを23度に置換
df['temp(℃)'][df['temp(℃)'] == 'rt'] = "23"
print(df['temp(℃)'].value_counts())

23    65
90    49
80     7
60     2
Name: temp(℃), dtype: int64


/var/folders/12/6r_lrg9d6kx949nw6gcq9ksh0000gn/T/ipykernel_89022/2448397563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp(℃)'][df['temp(℃)'] == 'rt'] = "23"


In [12]:
# カテゴリ値をダミー変数に変換
df = pd.get_dummies(df, columns=['R2','under','solvent'])
df.head(5)

,entry,R1-,organocatalyst,organocatalyst(mol%),temp(℃),time(h),収率(%),R2_C,R2_H,under_O2,under_air,solvent_CH3CN,solvent_DMF,solvent_MeCN,solvent_MeOH,solvent_toluene
0,1,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,24.0,81.0,0,1,1,0,0,0,0,0,1
1,2,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,12.0,14.0,0,1,1,0,0,0,0,0,1
2,3,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,24.0,74.0,0,1,1,0,0,0,0,0,1
3,4,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,12.0,15.0,0,1,1,0,0,0,0,0,1
4,5,[*]C1=CC=CC=C1,OC1=CC(OC)=CC=C1C(O)=O,5.0,90,24.0,90.0,0,1,1,0,0,0,0,0,1


In [13]:
# 重複確認
check_columns = df.columns.drop(['entry', '収率(%)'])

# 説明変数のみでみた時の重複削除
_df = df.drop(columns='entry').drop_duplicates(subset=check_columns).reset_index(drop=True)
X = _df.drop(columns=['収率(%)'])
y = _df['収率(%)']
_df.to_csv('exec_1114_前処理後.csv')

In [14]:
check_columns = df.columns.drop(['entry', '収率(%)'])
df.duplicated(subset=check_columns).value_counts()

False    123
dtype: int64

In [15]:
df.to_csv('./data/origin.csv')

# スケーリング

In [16]:
# 説明変数をスケーリング
# 目的変数は必要ない。
# https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re
from sklearn.preprocessing import StandardScaler
scaling_columns = ['organocatalyst(mol%)','temp(℃)','time(h)']
scaler = StandardScaler()
scaler.fit(df[scaling_columns])
df_scaled = pd.concat([df.drop(columns=scaling_columns), 
                           pd.DataFrame(scaler.transform(df[scaling_columns]),index=df.index , columns=scaling_columns)],
                           axis=1, join='inner')

print(df_scaled.shape)

(123, 16)


In [17]:
df_scaled.to_csv('./data/origin_scaled.csv')

# 記述子
https://qiita.com/ottantachinque/items/d7332bcf67b5844f5be3
### morganフィンガープリントの作成（Extended Connectivity Fingerprint）
子からある距離にある部分構造を数え上げていく，「circular substructures」と呼ばれるタイプのフィンガープリントです．radiusの値で距離を設定します．いわゆるECFP（Extended Connectivity Fingerprint）フィンガープリントに相当するものですが，探索距離の定義が異なる点に注意が必要です．
### morgan featureフィンガープリントの作成(Functional Connectivity Fingerprint）
FCFP（Functional Connectivity Fingerprint）様のフィンガープリントを用いたい場合にはuseFeatures=Trueを設定します．

### MACCS
全部で166の部分構造についての有無を調べ上げたもので，RDKit内の情報保持のために1ビット使っているため全部で167ビットのフィンガープリントになります．部分構造を有する場合は1が，ない場合は0が格納されています．

### RDkit記述子
一定の結合数に相当する原子と結合種類を格納する方法で，事前に部分構造を用意する必要がないことから，より柔軟に分子構造を表現することが可能です．

### MinHashフィンガープリント
「A probabilistic molecular fingerprint for big data settings」という論文で報告されている，ECFPと同様にCircular型のフィンガープリントになります

### Avalonフィンガープリント

### アトムペアフィンガープリントとトポロジカル二面角フィンガープリント
原子のアトムタイプとその最小結合距離を記録していきます．
アトムタイプには
原子の種類
結合する重原子の数
π電子数
https://future-chem.com/ap-dp-fingerprint/

### ドナーアクセプターペアフィンガープリント
ドナーアクセプターペアフィンガープリントでは「アトムタイプの情報」について

水素結合供与体
水素結合受容体
といった一般性のある「化学的性質」を情報として用いているフィンガープリントです

### mordred記述子の作成
https://github.com/mordred-descriptor


MolLogP : Crippenらによる原子ベースのLogP指標(LogPは水/オクタノール分配係数）
MolWt : 分子量
NumHAcceptors : 水素結合アクセプターの数
NumHDonors : 水素結合ドナーの数
NumHeteroatoms : へテロ元素の数
NumRotatableBonds : 回転可能な結合数
FractionCSP3 : 全炭素数におけるsp3炭素の割合
TPSA : トポロジカル極性表面積


In [18]:
BIT = 1024

In [27]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Avalon import pyAvalonTools
from mordred import Calculator, descriptors
from rdkit.Chem import rdMHFPFingerprint
encoder = rdMHFPFingerprint.MHFPEncoder()

def toFinger(df, columns, finger_method):
    df_copy = df.drop(columns=columns)
    for column in columns:
        fingerprints = toFingerFromSmiles(df[column],finger_method)
        column_names = list(map(lambda x: str(x)+'_'+column, range(len(fingerprints[0]))))
        df_copy = pd.merge(df_copy, pd.DataFrame(fingerprints,index=df.index, columns=column_names), left_index=True, right_index=True)
    return df_copy

def toFingerFromSmiles(series, method):
    mols = []
    for smile in series:
        if smile in ['-', 0]: smile = '' 
        mols.append(Chem.MolFromSmiles(smile))

    if (method == 'mordred'):
        calc_2D = Calculator(descriptors, ignore_3D=True) #2D記述子
        calc_3D = Calculator(descriptors, ignore_3D=False) #3D記述子
        fingerprints = calc_2D.pandas(mols,quiet=False)
        # fingerprint.index = labels
        return fingerprints
    else:
        fingerprints = []
        for mol_idx, mol in enumerate(mols):
            try:
                # listに直してる。
                if (method == 'morgan'):
                    fingerprint = [x for x in AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT)]
                elif (method == 'morgan_feature'):
                    fingerprint = [x for x in AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT,useFeatures=True)]
                elif (method == 'maccs'):
                    fingerprint = [x for x in AllChem.GetMACCSKeysFingerprint(mol)]
                elif (method == 'rdkit'):
                    fingerprint = [x for x in Chem.Fingerprints.FingerprintMols.FingerprintMol(mol)]
                elif (method == 'minhash'):
                    fingerprint = [x for x in encoder.EncodeMol(mol)]
                elif (method == 'avalon'):
                    fingerprint = [x for x in pyAvalonTools.GetAvalonFP(mol)]
                elif (method == 'atom'):
                    fingerprint = [x for x in Chem.AtomPairs.Pairs.GetAtomPairFingerprint(mol)]
                elif (method == 'donor'):
                    fingerprint = [x for x in Chem.AtomPairs.Sheridan.GetBPFingerprint(mol)]
                else:
                    print('method error')
                fingerprints.append(fingerprint)
            except Exception as e:
                print("Error", mol_idx)
                break
        return fingerprints

In [20]:
finger_methods = [
    'morgan',
    'morgan_feature',
    'maccs',
    'rdkit',
    'minhash',
    'avalon',
    'atom',
    'donor',
]

# 記述子変換 ⇒ 分散が0 or 80%異常が同じ値の情報を削除
説明しきれていないため共線性の恐れがある
VarianceThresholdで特徴選択

In [24]:
from sklearn.feature_selection import VarianceThreshold
def removeNoImpactData(df):
    X = df.drop(columns=['entry','収率(%)'])
    select = VarianceThreshold()
    X_new = select.fit_transform(X)
    df_new = pd.concat([
        pd.DataFrame(X_new, columns=np.array(df)[select.get_support()==True]),
        df['収率(%)']
    ],axis=1)
    return df_new

# finger print変換
- method1 全てのデータを使用
- 

In [30]:
# 1. 全てのデータを使用
finger_columns = ['R1-', 'organocatalyst']
for finger_method in finger_methods:
    print(finger_method)
    df_method1 = toFinger(df_scaled, finger_columns,finger_method)
    removeNoImpactData(df_method1).to_csv(f'data/method1/${finger_method}.csv')




morgan
Error 65


[02:24:20] SMILES Parse Error: unclosed ring for input: '[*]C1=CC=C(C)C=C2'
[02:24:20] SMILES Parse Error: unclosed ring for input: '[*]C1=CC=C(C)C=C3'
[02:24:20] SMILES Parse Error: unclosed ring for input: '[*]C1=CC=C(OC)C=C2'


ValueError: Length of values (65) does not match length of index (123)